TODO
- [ ] wordcloud
- [ ] graphs, charts, statistics


In [1]:
import bs4
import requests
import pandas as pd
import re
import fandom
import random
import time
import pickle

In [2]:
# 

LOAD = False
SAVE = True

In [3]:
domain_address = 'https://rickandmorty.fandom.com/'
all_pages_subpage = '/wiki/Special:AllPages'

In [4]:
# next_page_link = domain_address + all_pages_subpage


# list_of_pages = []

# # collect all links
# count = 0

# re_next = re.compile(r'Next page')

# while count < 5:
#     response = requests.get(next_page_link)
#     parsed = bs4.BeautifulSoup(response.text)

#     div_with_list_of_links = parsed.find_all('div', {'class':'mw-allpages-body'})[0]

#     for a_element in div_with_list_of_links.find_all('a', href=True):
#         list_of_pages.append(a_element['href'])
    
#     next_page_div = parsed.find_all('div', {'class':'mw-allpages-nav'})[0]
#     page_routes = next_page_div.find_all('a', href=True)
#     for page_route in page_routes:
#         if re.search(re_next, page_route.contents[0]):
#             next_page_route = page_route['href']
#         print(next_page_routes)
#     next_page_link = None
#     next_page_link =domain_address +  next_page_div.find_all('a', href=True)[-1]['href']
#     print(next_page_link)

#     count +=1
    

Scrapping the links of all subpages in a fandom

In [5]:
if not LOAD:

    next_page_link = domain_address + all_pages_subpage
    list_of_pages = []

    # collect all links
    count = 0

    re_next = re.compile(r'Next page')

    while count < 50:
        time.sleep(random.randrange(3, 7)*0.1)
        response = requests.get(next_page_link)
        parsed = bs4.BeautifulSoup(response.text)

        div_with_list_of_links = parsed.find_all('div', {'class':'mw-allpages-body'})[0]

        for a_element in div_with_list_of_links.find_all('a', href=True):
            list_of_pages.append(domain_address + a_element['href'])
        
        next_page_div = parsed.find_all('div', {'class':'mw-allpages-nav'})[0]
        page_routes = next_page_div.find_all('a', href=True)
        next_page_route = None
        for page_route in page_routes:
            if re.search(re_next, page_route.contents[0]):
                next_page_route = page_route['href']
        if next_page_route is None:
            break
        next_page_link =domain_address +  next_page_route
        print(next_page_link)

        count +=1
        

https://rickandmorty.fandom.com//wiki/Special:AllPages?from=Charred+Morty
https://rickandmorty.fandom.com//wiki/Special:AllPages?from=Female+Jerry
https://rickandmorty.fandom.com//wiki/Special:AllPages?from=Jerry+Smith+%28C-137%29
https://rickandmorty.fandom.com//wiki/Special:AllPages?from=Mr.+Goldenfold+%28Wasp+Universe%29
https://rickandmorty.fandom.com//wiki/Special:AllPages?from=Rainbow+Suspenders+Guy
https://rickandmorty.fandom.com//wiki/Special:AllPages?from=Samantha
https://rickandmorty.fandom.com//wiki/Special:AllPages?from=The+Non-Canonical+Adventures


In [6]:
if SAVE:

    with open('./data/list-of-pages.txt', 'w') as outfile:
        for page in list_of_pages:
            outfile.write(f'{page}\n')





Scrapping text from all pages in the fandom

In [7]:
# Używam jakiegoś scrappere którego ktoś napisał dla fandom, ale jest trochę wolny

fandom.set_wiki('rickandmorty')
fp = fandom.page(title='2001:_A_Space_Odyssey')

In [8]:
fp.plain_text

"2001: A Space Odyssey\n2001: A Space Odyssey is a claymation short from Rick and Morty's The Non-Canonical Adventures.\nOverview\nThis short is a parody of the 1968 film, 2001: A Space Odyssey.\nPlot\nMorty is seen in a Space Shuttle wearing a Space suit. He asks Rick to open the door. Rick says that he can't do that. Morty asks what is the problem but Rick replies that this conversation can serve no purpose and tells Morty Goodbye in multiple languages.\nCharacters\nMorty Smith\nRick Sanchez (voice)"

In [9]:
def get_text_from_page(link:str)-> str:
    response = requests.get(link)
    parsed = bs4.BeautifulSoup(response.text)

    title = link.split('/')[-1]
    page = fandom.page(title=title)
    
    content = page.plain_text
    
    return content
    

In [10]:
get_text_from_page(list_of_pages[-2])

'Zombodians\nZombodians are an alien species that appears in the Rick and Morty universe. They appear throughout the show. One such member of this race is Gar Gloonch\nAppearance\nZombodians are shorter in height that the average human, ranging around three feet tall. They have a mix of light blue and seafoam green skin with two eyes with eyestalks that bend down to the face. Their blood color is orange.\nSection heading\nWrite the second section of your page here.'

In [11]:
from tqdm import tqdm

In [12]:
articles = []
for page_link in tqdm(list_of_pages):
    try:
        try:
            try:
                time.sleep(random.randrange(50, 300)/1000)
                content = get_text_from_page(page_link)
                articles.append((page_link, content))
            except ValueError as ve:
                print(f'{page_link} article is empty')
        except fandom.error.PageError as pe:
            print(f'cannot find: {page_link}')
    except Exception as e:
        print('unexpected error has occurred.')


  0%|          | 1/2705 [00:01<1:01:32,  1.37s/it]

cannot find: https://rickandmorty.fandom.com//wiki/%22Lawyer%22_Morty


  0%|          | 2/2705 [00:02<1:07:31,  1.50s/it]

cannot find: https://rickandmorty.fandom.com//wiki/1%25_Visible


  0%|          | 8/2705 [00:16<1:40:15,  2.23s/it]

...

 99%|█████████▉| 2690/2705 [1:35:07<00:29,  1.99s/it]

cannot find: https://rickandmorty.fandom.com//wiki/Yummy%27_Yums


100%|█████████▉| 2692/2705 [1:35:10<00:24,  1.87s/it]

cannot find: https://rickandmorty.fandom.com//wiki/Zarbadar%27s_Mytholog


100%|██████████| 2705/2705 [1:35:38<00:00,  2.12s/it]


- remove site navigation 
- remove non alphanumeric characters

In [13]:
df_articles = pd.DataFrame(articles, columns=['link', 'content'])


In [15]:
if SAVE:
    df_articles.to_csv('df_articles.csv')

In [16]:
df_articles.content.iloc[0]

'1950s Guy\n1950s Guy is a minor character from The Real Animated Adventures of Doc and Mharti\nAppearance\n1950s Guy wears a white t-shirt with the year "1955" written on it, with a black vest worn over it. He wears blue jeans and a black belt.\nHe is seen carrying a bag of tree seeds.\nHistory\n1950s guy was a character who appeared in The Real Animated Adventures of Doc and Mharti. In one of the shorts, Mharti McDonhalds is flying a kite, but it unfortunately gets stuck in a tree.\nDoc and Mharti travel back in time to the 1950s to prevent the tree from ever being planted. When the 1950s Guy arrives to plant the seed of the tree, Mharti licks Doc\'s balls. Once 1950s Guy sees this, he starts screaming in pain and eventually fades from the timeline, and is unable to plant the tree seed.\nLater, Mharti starts fading from the timeline in the present day, revealing that 1950\'s guy must be Mharti\'s great-great-great-great-great grandfather.\nv • e Rick and Morty Characters \nMain chara

Now, as dataframe is clean, we perform NLP processing

In [17]:
from nltk.stem import WordNetLemmatizer
from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize


In [18]:
def preprocess(raw_text:str):
    text = raw_text.lower()

    text = re.sub('\\nv(.|\n)*', ' ', text)
    text = re.sub('\\n', ' ', text)
    text = re.sub('[^\w\s]', ' ', text)
    # text = re.sub('site navigation.*', '', text)
    text = re.sub(' s ', '', text)

    # tokenize
    tokens = word_tokenize(text)

    # lemmatize
    lemmatizer = WordNetLemmatizer()
    tokens = [lemmatizer.lemmatize(token) for token in tokens]

    # drop stopwords
    # english_stopwords = stopwords.words('english')
    
    return ' '.join(tokens)

In [19]:
df_articles.content = df_articles.content.apply(preprocess)

In [20]:
if SAVE:
    df_articles.to_csv('df_articles_processed.csv')

Now the dataframe is ready to be processed by TfIdf

In [21]:
from sklearn.feature_extraction.text import TfidfVectorizer

vectorizer = TfidfVectorizer(stop_words='english', ngram_range=(1,2))
X = vectorizer.fit_transform(df_articles.content)

In [22]:
X.toarray().shape

(2332, 184307)

In [30]:
df = pd.DataFrame(X.toarray(),
    columns = vectorizer.get_feature_names_out())

In [31]:
df.head()

,00,00 10,00 12,00 50,00 clock,00 einstienoffice,00 garage,00 hand,00 hawaii,00 hospital,...,zoo prostate,zoo wa,zookeeper,zookeeper annie,zoom,zoom surrounded,zooming,zooming rick,árboles,árboles mentirosos
0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.000000,0.000000,0.0,0.0,0.0,0.0
1,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.000000,0.000000,0.0,0.0,0.0,0.0
2,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.000000,0.000000,0.0,0.0,0.0,0.0
3,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.000000,0.000000,0.0,0.0,0.0,0.0
4,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.095932,0.095932,0.0,0.0,0.0,0.0


We can sort the idf vectors by frequency, to see which terms are the least informative.

- TODO: present the following using `wordcloud`?


In [23]:
idf_df = pd.DataFrame(
    vectorizer.idf_,
    index=vectorizer.get_feature_names_out(),
    columns=['idf_weights']
)

idf_df.sort_values(by=['idf_weights']).head(20)


,idf_weights
morty,1.274482
rick,1.354901
wa,1.573318
navigation,1.779496
site,1.790775
site navigation,1.805054
rick morty,1.864301
episode,1.969323
trivia,2.034690
ha,2.070299


In [24]:
# compute cosine similarities  of each cluster]
from sklearn.metrics.pairwise import cosine_similarity

query = 'birdperson'

query_clean = preprocess(query)

query_vector = vectorizer.transform([query_clean])

In [25]:
query_vector.toarray().shape

(1, 184307)

In [26]:
cosine= cosine_similarity(query_vector, X)
similarities = pd.DataFrame({'link': df_articles.link.iloc[:], "cosine": cosine.flatten()})
similarities = similarities.sort_values(by='cosine',ascending=False)

In [27]:
similarities.head(10)

,link,cosine
171,https://rickandmorty.fandom.com//wiki/Bird_Person,0.499120
175,https://rickandmorty.fandom.com//wiki/Birdperson,0.499120
1357,https://rickandmorty.fandom.com//wiki/Phoenixp...,0.499120
1739,https://rickandmorty.fandom.com//wiki/Ricktern...,0.394130
646,https://rickandmorty.fandom.com//wiki/Freedom_...,0.245990
1665,https://rickandmorty.fandom.com//wiki/Rick_and...,0.211692
2031,https://rickandmorty.fandom.com//wiki/Tammy,0.188464
2034,https://rickandmorty.fandom.com//wiki/Tammy_Gu...,0.188464
2033,https://rickandmorty.fandom.com//wiki/Tammy_Gu...,0.188464
2032,https://rickandmorty.fandom.com//wiki/Tammy_Gu...,0.188464


### Dimensionality reduction

we can also experiment with dimensionality reduction to retreieve key concepts present in the fandom.

- SVD
- NMF